# Node Graph Map Calculations
This calculations presents as part of 3D Interactive Assets component where user can quickly navigate to the points that imitate the real model hotspots by click one of the node.

## Real Model Node

$Vp1$, $Vp2$ and $Vp3$ represents as real vector positions from the models position

In [ ]:
from sage.all import *
from sage.plot.plot3d.shapes2 import text3d
pointHs = 5.0
originV = vector([0,0,0])
Vp1 = vector([-155.0, 50., 30.])
Vp2 = vector([55.0, 10., 10.])
Vp3 = vector([-155.0, 20., 80.])

Create empty scene

In [ ]:
scene = sphere([0.0,0.0,0.0], size=0.0, opacity=0.0)

Create Sphere based on real locations hotspot points & line connected from origin to each location of $Vp1$, $Vp2$ and $Vp3$

In [ ]:
origin = text3d('Origin', tuple(originV))

offsetLabel = vector([0.0,0.0,10.0])
hs1Label = text3d('HS1', tuple(Vp1 + offsetLabel))
hs2Label = text3d('HS2', tuple(Vp2 + offsetLabel))
hs3Label = text3d('HS3', tuple(Vp3 + offsetLabel))

hs1 = sphere(Vp1, size=pointHs)
hs2 = sphere(Vp2, size=pointHs)
hs3 = sphere(Vp3, size=pointHs)

line1 = line3d([Vp1, originV], arrow=True)
line2 = line3d([Vp2, originV], arrow=True)
line3 = line3d([Vp3, originV], arrow=True)

scene += hs1 + hs2 + hs3 + hs1Label + hs2Label + hs3Label + origin + line1 + line2 + line3

scene.show(frame=False)

## Node Graph Map

Calculate Unit Vector based on the location no matter if the real location is in negative value. $\mathbf{\hat{v}}$ represents as Unit Vector and $\mathbf{|v|}$ represents as magnitude

$$
\mathbf{\hat{v} = \frac{v}{|v|}}
$$

$$
\mathbf{\hat{v} = \frac{Vx}{|v|} + \frac{Vy}{|v|}}
$$

$$
\mathbf{|v| = \sqrt{Vx^2 + Vy^2}}
$$

In [ ]:
def unitvector(v):
    """
    To Calculate Unit Vector
    :param v: Vector
    :return: vector
    """
    uv = sqrt(sum(val**2 for val in v))
    return vector([originalVal/uv for originalVal in v])

In [ ]:
from sage.repl.ipython_kernel.widgets_sagenb import slider, Color

UVp1 = unitvector(Vp1)
UVp2 = unitvector(Vp2)
UVp3 = unitvector(Vp3)

offsetLabeluv = vector([0.0,0.0,0.1])

@interact
def run_node_graph(size_division=slider(vmin=1.0, vmax=1000.0, default=100., step_size=1.0)):
    scene_node = sphere([0.0,0.0,0.0], size=0.0, opacity=0.0)

    uvhs1= sphere(UVp1, size=pointHs/size_division)
    uvhs2= sphere(UVp2, size=pointHs/size_division)
    uvhs3= sphere(UVp3, size=pointHs/size_division)

    labeluvhs1 = text3d('HS1', tuple(UVp1 + offsetLabeluv))
    labeluvhs2 = text3d('HS2', tuple(UVp2 + offsetLabeluv))
    labeluvhs3 = text3d('HS3', tuple(UVp3 + offsetLabeluv))

    lineuvhs1 = line3d([UVp1, originV], arrow=True)
    lineuvhs2 = line3d([UVp2, originV], arrow=True)
    lineuvhs3 = line3d([UVp3, originV], arrow=True)

    sphere_visual = sphere(vector([0.,0.,0.]), size=1., opacity=0.5)

    scene_node += uvhs1 + uvhs2 + uvhs3 + lineuvhs1 + lineuvhs2 + lineuvhs3 + labeluvhs1 + labeluvhs2 + labeluvhs3 + sphere_visual

    scene_node.show()

## Animate Node Graph
Now let's test animate the node graph when hotspot changes just to experiment if the node vector is always return length of $1$

In [ ]:
from sage.all import *
import random

# ---------- helpers ----------
def generate_nonzero_vector(scale=100.0, eps=1e-9, max_attempts=1000):
    while True:
        # random.uniform(a, b) gives float in [a, b]
        v = vector([random.uniform(-scale, scale) for _ in range(3)])
        if v.norm() > eps:  # skip origin (or near-origin)
            return v


def next_random_far_from(v_last, scale=100.0, min_step=5.0):
    while True:
        v = generate_nonzero_vector(scale)
        if (v - v_last).norm() >= min_step:
            return v

def lerp(a, b, t):
    return (1 - t) * a + t * b

# ---------- build frames ----------
def build_animation(num_cycles=8, steps_per_cycle=15, scale=100.0,
                    sphere_size=2.0, show_trail=True, show_sphere=False):
    frames = []

    # start at origin once
    start1 = start2 = start3 = vector([0.0, 0.0, 0.0])
    # first targets
    r1 = generate_nonzero_vector(scale)
    r2 = generate_nonzero_vector(scale)
    r3 = generate_nonzero_vector(scale)

    # trails (as connected lines)
    trail1 = [start1]
    trail2 = [start2]
    trail3 = [start3]

    for cycle in range(num_cycles):
        for i in range(steps_per_cycle):
            t = i / float(steps_per_cycle - 1)  # includes 0 and 1
            p1 = lerp(start1, r1, t)
            p2 = lerp(start2, r2, t)
            p3 = lerp(start3, r3, t)

            G = sphere((0, 0, 0), size=1.0, opacity=0.3)  # reference origin

            # current positions
            G += sphere(p1, size=sphere_size, color='blue')
            G += sphere(p2, size=sphere_size, color='blue')
            G += sphere(p3, size=sphere_size, color='blue')

            # Line from origin
            line_r1 = line3d([vector([0.,0.,0.]), p1], arrow=True)
            line_r2 = line3d([vector([0.,0.,0.]), p2], arrow=True)
            line_r3 = line3d([vector([0.,0.,0.]), p3], arrow=True)

            G += line_r1 + line_r2 + line_r3

            # show trails up to current point
            if show_trail:
                G += line3d(trail1 + [p1], color='lightblue', thickness=2)
                G += line3d(trail2 + [p2], color='lightblue', thickness=2)
                G += line3d(trail3 + [p3], color='lightblue', thickness=2)

            frames.append(G)

        # end of cycle: commit endpoints to trails and advance start/targets
        start1, start2, start3 = r1, r2, r3
        trail1.append(start1); trail2.append(start2); trail3.append(start3)
        r1 = next_random_far_from(start1, scale=scale)
        r2 = next_random_far_from(start2, scale=scale)
        r3 = next_random_far_from(start3, scale=scale)

    return frames

# ---------- run ----------
frames = build_animation(num_cycles=10, steps_per_cycle=20, scale=100.0,
                         sphere_size=2.0, show_trail=False)
A = animate(frames, axes=True).interactive(online=True)
A.show()
